# Test Affinity and comp algorithm on liquid
- this algorithm is usually fast and accurate
- but it can fail to converge, causing major problems


In [ ]:
import numpy as np
from os import path
import pandas as pd
import scipy.optimize as opt
from scipy import optimize
import scipy.linalg as lin
import scipy as sp
import sys
import sympy as sym

from collections import OrderedDict as odict

import matplotlib.pyplot as plt

import sunkenhull as hull
from thermoengine.model import GeoCompDB, SysComp

Required ENKI modules (ignore the error message from Rubicon running under Python 3.6+)

In [ ]:
from thermoengine import coder, core, phases, model, equilibrate

In [ ]:
modelDB = model.Database(database='Berman')

## Assume MORB Bulk Composition


In [ ]:
compDB = GeoCompDB()

In [ ]:
sysID = 'MORB_primitive'
CO2='none'
# H2O='none'
H2O = 'input'
syscomp = compDB.get_syscomp(
    sysID, components='oxides',CO2=CO2,H2O=H2O)
display(syscomp.wt_comp())
display(syscomp.mol_comp('oxides'))

In [ ]:
elem_comp = syscomp.mol_comp(components='elems')
sys_elems=elem_comp.columns.values
bulk_comp = elem_comp.values[0]

## P/T conditions selected to involve many phases present

In [ ]:
T = 1175+273.15
T= 1600+273
P = 1e3

## Initialize assemblage w/ pure liquid
- if omnicomponent phase is present this will produce good initial guess for chempot
- assume pure liquid for MORB composition system


In [ ]:
mol_oxides=syscomp.mol_comp(components='oxides')
mol_oxides['CO2']=0
mol_oxides.values.squeeze()

In [ ]:
liq = modelDB.get_phase('Liq')

In [ ]:
mol_endmem = liq.calc_endmember_comp(mol_oxides.values.squeeze(),method='intrinsic')
mol_endmem/=mol_endmem.sum()

In [ ]:
chempot = liq.chem_potential(T, P,  mol=mol_endmem).squeeze()

In [ ]:
dG = liq.gibbs_energy(T, P, mol=mol_endmem)-chempot.dot(mol_endmem)
dG

In [ ]:
# lin.lstsq()

In [ ]:
liq_elem_comp = pd.DataFrame(liq.props['element_comp'],index=liq.endmember_names, 
                             columns=SysComp.PERIODIC_ORDER)[sys_elems]
liq_elem_comp


## verify that liquid composition calculated correctly

In [ ]:
elem_diff = liq_elem_comp.T.dot(mol_endmem)-elem_comp
elem_diff.abs()<1e-10

In [ ]:
chempot_elems = lin.lstsq(liq_elem_comp, chempot)[0]
chempot_elems

In [ ]:
mu = chempot.copy()
mu[mol_endmem==0]=0
mu

In [ ]:
noise = .3
# noise=0

In [ ]:
X_init = mol_endmem*np.exp(noise*np.random.randn(mu.size))
X_init[X_init<0] = 0
X_init[X_init>1] = 1
X_init[mol_endmem==0] = 0

In [ ]:
# X_init /= X_init.sum()

In [ ]:
%%timeit
liq.chem_potential(T, P,  mol=X_init).squeeze()

In [ ]:
%%timeit
liq.gibbs_energy(T, P,  mol=X_init, deriv={'dmol':1}).squeeze()

## Benchmark calc. times
- compare legacy algo and new direct algo
- compare with hot start

### Legacy algorithm
- fails to completely converge

In [ ]:
%%timeit
A0, X0 = liq.affinity_and_comp_legacy(T, P, mu, debug=False)

## Updated Direct Algorithm
- converges w/in machine precision
- fewer iterations and much faster

### Cold-start
- converges w/in machine precision
- ~30x faster

In [ ]:
converge_method='approx'

converge_method='lstsq'
#converge_method='direct'
site_m = liq.exchange_equil._est_site_mult(T, P)

In [ ]:
%%timeit
A, X = liq.affinity_and_comp(T, P, mu, converge_method=converge_method, site_m=site_m)

### Warm-start
- start w/ noisy initial guess for composition
- approx same runtime as cold-start (depending on phase, just a bit faster)
- perfect convergence

In [ ]:
%%timeit
A, X = liq.affinity_and_comp(T, P, mu, X_init=X_init, converge_method=converge_method, site_m=site_m)

### Hot-start
- initialized with very good guess
- is 2x faster again

In [ ]:
noise = .01
# noise=0

In [ ]:
X_init = mol_endmem*np.exp(noise*np.random.randn(mu.size))
X_init[X_init<0] = 0
X_init[X_init>1] = 1
X_init[mol_endmem==0] = 0

In [ ]:
%%timeit
A, X = liq.affinity_and_comp(T, P, mu, X_init=X_init, converge_method=converge_method, site_m=site_m)

### Perfect start
- 3x faster again
- only attained currently using perfect initial guess for composition
- provides limiting case for speed

In [ ]:
%%timeit
A, X = liq.affinity_and_comp(T, P, mu, X_init=mol_endmem, converge_method=converge_method, site_m=site_m)

In [ ]:
%load_ext snakeviz

In [ ]:
%%snakeviz
A, X = liq.affinity_and_comp(T, P, mu, converge_method='lstsq', site_m=site_m)